In [1]:
import warnings
warnings.filterwarnings("ignore") 

import numpy as np
import pandas as pd

import nltk    #Natural language processing tool-kit
nltk.download('stopwords')
  
from nltk.corpus import stopwords                   #Stopwords corpus

from keras.datasets import imdb
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patricia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


# Données

In [55]:
(X_train, Y_train), (X_test, Y_test)  = imdb.load_data()

Vérification des dimensions

In [62]:
print(" Taille X_train:",np.shape(X_train),"\n",
      "Taille X_test:",np.shape(X_test),"\n",
      "Taille Y_train:",np.shape(Y_train),"\n",
      "Taille Y_test:",np.shape(Y_test))

 Taille X_train: (25000,) 
 Taille X_test: (25000,) 
 Taille Y_train: (25000,) 
 Taille Y_test: (25000,)


### Labels

In [130]:
X = pd.DataFrame(np.concatenate((X_train, X_test)))
y = pd.DataFrame(np.concatenate((Y_train, Y_test)))

In [28]:
print(" Labels :" ,np.unique(y))

 Labels : [0 1]


Il s'agit d'un problème de classification binaire.

In [93]:
np.shape(Y_train)[0]

25000

In [106]:
Y_train[24999]

0

### Class-Balancing 

On écrit une fonction permettant de vérifier que les classes sont bien équilibrées dans le train et le test:

In [117]:
def classBalancing(labelVector):
    nbr_0  = 0
    nbr_1 = 0
    seq = np.shape(labelVector)[0]
    i = 0
    for i in range(seq):
        if labelVector[i] == 0:
            nbr_0 = nbr_0 + 1
        else:
            nbr_1 = nbr_1 + 1 
    return nbr_0,nbr_1

In [119]:
print("Nombre de 0 et de 1 dans le train :",classBalancing(Y_train))

Nombre de 0 et de 1 dans le train : (12500, 12500)


In [120]:
print("Nombre de 0 et de 1 dans le test :",classBalancing(Y_test))

Nombre de 0 et de 1 dans le test : (12500, 12500)


# Hashage - déHashage

On commence par remarquer que les textes ont déja été hashés : 

In [121]:
X_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

On écrit une fonction pour reconstruite le texte (avec les mots les plus fréquents uniquement) :

In [211]:
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    print(listOfKeys)

Verifions que ça fonctionne pour le sixième commentaire:

In [217]:
for word_num in X_train[5]:
    getKeysByValue(imdb.get_word_index(),word_num)

['the']
['effort']
['still']
['been']
['that']
['usually']
['makes']
['for']
['of']
['finished']
['sucking']
['ended']
["cbc's"]
['an']
['because']
['before']
['if']
['just']
['though']
['something']
['know']
['novel']
['female']
['i']
['i']
['slowly']
['lot']
['of']
['above']
['freshened']
['with']
['connect']
['in']
['of']
['script']
['their']
['that']
['out']
['end']
['his']
['deceptively']
['i']
['i']


Plutôt que d'effectuer la transformation entier par entier, transformons tout un vecteur (donc toute une ligne de notre X_train par exemple)

In [198]:
def vectorToSentence(Vector):
    for word_num in Vector:
        getKeysByValue(imdb.get_word_index(),word_num)

"Derniere" étape : on transforme les données pleines de chiffres illisibles en mots : 

In [203]:
def intToReviews(data):
    k = 0
    for k in range (np.shape(data)[0]):
        data_txt = vectorToSentence(data[k])

# Bag of words

Maintenant on va créer un dictionnaire avec tous les mots de tous les commentaires de nos données d'apprentissage. Puis on va compter le nombre d'apparition de chaque mot du dictionnaire dans chacune des phrases des données

Les données sont trés volumineuses, ne pouvant pas compiler sur mon ordi, je vous donne quand même le code en esperant qu'il n'y a pas trop d'erreurs.

In [ ]:
X_train_txt = intToReviews(X_train)

In [223]:
count_vect = CountVectorizer() # transforme text en vecteur
X_counts = count_vect.fit_transform(X_train_txt)
dictionary_sub   = count_vect.get_feature_names()
print(np.shape(dictionary_sub))

On écrit une fonction qui prend en arguments la vectorization et les données d'apprentissage.
On crée un dictionnaire à partir de ces données.
on crée un data frame à partir de la matrice sparse qui compte la présence de chaque mot du dictionnaire dans chaque phrase des données d'apprentissage : word_counts.

In [214]:
count_vect = CountVectorizer()

def most_frequent_words(count_vect,data):  
    
    data_counts = count_vect.fit_transform(data)
    dictionary   = count_vect.get_feature_names()
    print("Taille du dictionnaire : ",len(dictionary))
    
    word_counts = pd.DataFrame(data_counts.toarray(),columns = dictionary)
    word_total_counts = pd.DataFrame()
    word_total_counts['count'] = word_counts.sum(axis=0)
    word_total_counts['word'] = dictionary
    
    word_total_counts_sorted_top  = word_total_counts.sort_values(by="count",ascending = False).iloc[0:20]
    
    plt.figure(figsize=(15,8))
    sns.barplot(x = word_total_counts_sorted_top.index,y=word_total_counts_sorted_top['count'])
    plt.xticks(rotation=90,fontsize=10)
    plt.show()
    
    return {'data_counts' : data_counts, 'word_total_counts' :word_total_counts, 'dictionary' : dictionary}

# Préparation des données

In [237]:
stop = set(stopwords.words('english')) 
print(stop)

{"doesn't", 'because', 'both', "hadn't", 'did', 'out', 'it', 'your', "that'll", 'she', 'its', 'these', 'by', "it's", "should've", 'further', 'his', 'm', 'have', "couldn't", "you're", 'doesn', 'up', 'can', 'with', 'which', 'own', 'the', "shan't", "you'll", "wouldn't", 'all', 've', 'being', 'won', 'during', 'do', "won't", 'whom', 'an', 'once', 'and', 'into', 're', 'too', 'each', 'theirs', 'we', 'over', 'does', 'hasn', 'was', 'here', 'not', 'our', 'how', 'than', "she's", 'wouldn', 'very', 'but', 'through', 'd', 'me', 'again', 'haven', 'now', 'needn', 'just', 'wasn', "wasn't", 'them', 'they', "aren't", 'such', 'in', 'yours', 'didn', 'down', 'been', 'yourself', 'few', 'from', 'there', 'at', 'what', 'no', 'isn', 'mightn', 'why', 'while', 'yourselves', 'off', 'having', 'under', 'about', 'most', 'don', "needn't", 'had', 'aren', 'myself', 'itself', 'couldn', 'to', 'any', 'you', "hasn't", 'were', 'before', 'below', 'where', 'is', 'he', 'ours', 'other', 'should', 'then', 'hadn', 't', 'are', 'am',

In [ ]:
count_vect_sw = CountVectorizer(stop_words = stop)
most_frequent_words_sw = most_frequent_words(count_vect_sw,X_train_txt) # enleve les mots non pertinents

In [ ]:
stop.add('movie')
stop.add('film')

### Visualisation du corpus

On calcule l'histogramme : proportion de chacun des mots du dictionnaire $ h_w(d) $

In [ ]:
most_frequent = np.array(most_frequent_words_sw['data_counts'].toarray())
hist = most_frequent / most_frequent.sum(axis=1)[:,None]

In [ ]:
hist[np.argsort(df['label']),:]

In [ ]:
plt.figure(figsize=(15,8))
plt.imshow(hist[np.argsort(df['label']),:])
#plt.colorbar()

### TF_IDF

In [ ]:
transformer = TfidfTransformer(smooth_idf=False)
transformer

In [ ]:
tf_idf_transf = transformer.fit_transform(most_frequent_words_sw['data_counts'])

In [ ]:
plt.figure(figsize=(15,8))
plt.imshow(tf_idf_transf.toarray()[np.argsort(df['label']),:])
plt.colorbar()

# Apprentissage supervisé

### Naive Bayes

Sur le count

In [226]:
NB_counts = MultinomialNB()
NB_counts.fit(X=most_frequent_words_sw['data_counts'],y=df['label'])
accuracy_score(NB_counts.predict(most_frequent_words_sw['data_counts']),df['label'])

Sur le TF-IDF

In [ ]:
NB_tfidf = GaussianNB()
NB_tfidf.fit(X=tf_idf_transf.toarray(),y=df['label'])
accuracy_score(NB_tfidf.predict(tf_idf_transf.toarray()),df['label'])

### Regression logistique

Sur le count

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr_counts = LogisticRegression(C=c)
    lr_counts.fit(X=most_frequent_words_sw['data_counts'],y=df['label'])
    print ("Accuracy for C= %s: %s"  %(c, accuracy_score(lr_counts.predict(most_frequent_words_sw['data_counts']),df['label'])))
    

In [ ]:
Sur le TF-IDF

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr_tfidf = LogisticRegression(C=c)
    lr_tfidf.fit(X=most_frequent_words_sw['data_counts'],y=df['label'])
    print ("Accuracy for C= %s: %s"  %(c, accuracy_score(lr_tfidf.predict(tf_idf_transf.toarray()),df['label'])))
    